In [ ]:
#importamos librerias

import pandas as pd
import numpy as np
import datetime
from decimal import Decimal
import json
import os

In [ ]:
!pip install sweetviz

In [ ]:
import sweetviz as sv

In [ ]:
#leemos el archivo checkin de la carpeta yelp
check = pd.read_json('/content/drive/MyDrive/Yelp/checkin.json', lines= True )

In [ ]:
#un pequeño analisis con .describe()
check.describe()

,business_id,date
count,131930,131930
unique,131930,131930
top,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
freq,1,1


In [ ]:
#revisamos que no hayan nulos y duplicados
check.isnull().sum()


0

In [ ]:
check.duplicated().sum()

In [ ]:
check

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."
...,...,...
131925,zznJox6-nmXlGYNWgTDwQQ,"2013-03-23 16:22:47, 2013-04-07 02:03:12, 2013..."
131926,zznZqH9CiAznbkV6fXyHWA,2021-06-12 01:16:12
131927,zzu6_r3DxBJuXcjnOYVdTw,"2011-05-24 01:35:13, 2012-01-01 23:44:33, 2012..."
131928,zzw66H6hVjXQEt0Js3Mo4A,"2016-12-03 23:33:26, 2018-12-02 19:08:45"


In [ ]:
# Define una función para convertir un valor de tiempo en milisegundos a una fecha
def convertir_a_fecha(timestamp):
    timestamp /= 1000  # Convertir milisegundos a segundos
    fecha = datetime.datetime.utcfromtimestamp(timestamp)
    return fecha


Conversiones

In [ ]:
#importamos los archivos de drive para que sean accesibles
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#guardamos la direccion de la carpeta que vamos a iterar
root_folder = "/content/drive/MyDrive/Google_Maps/reviews-estados"

In [ ]:
#seleccionamos todas las carpetas que hay en la carpeta reviews-estados
all_folders = os.listdir(root_folder)

In [ ]:
# Carpeta para almacenar los archivos CSV resultantes
carpeta_destino = '/content/csvestados'

# Crear la carpeta 'csvestados' si no existe
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)
#se guardan los nombres de las columnas que deseamos eliminar
a = ['resp', 'pics']
#recorriendo todas las carpetas una por una se leen los archivos json que contienen
for folder in all_folders:
    folder_path = os.path.join(root_folder, folder)
    json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

    if json_files:
        dataframes = []
        for json_file in json_files:
            json_path = os.path.join(folder_path, json_file)
            df = pd.read_json(json_path, lines=True)
            dataframes.append(df)
        #despues de la lectura, se hacen las transformaciones necesarias
        combined_df = pd.concat(dataframes, ignore_index=True)
        combined_df = combined_df.drop(a, axis=1)
        combined_df = combined_df.drop_duplicates(keep="first")
        combined_df['user_id'] = combined_df['user_id'].apply(Decimal)
        combined_df['time'] = combined_df['time'].apply(convertir_a_fecha)

        # Nombre del archivo de destino
        archivo_destino = os.path.join(carpeta_destino, f"{folder}.csv")

        # Guardar el DataFrame en la carpeta de destino
        combined_df.to_csv(archivo_destino, index=False, escapechar='\\')

print(f"Los archivos CSV se han almacenado en la carpeta '{carpeta_destino}'.")

Los archivos CSV se han almacenado en la carpeta '/content/csvestados'.


In [ ]:
#guardamos las direcciones de las carpetas donde estan los csv y donde queremos guardar los reportes automaticos
csv_folder = "/content"
reports_folder = "/content/Reviews_estados"

In [ ]:
#iteramos por cada archivo csv para que genere un reporte automatico por cada uno
for csv_file in os.listdir(csv_folder):
    if csv_file.endswith(".csv"):
        csv_path = os.path.join(csv_folder, csv_file)

        # Leer el archivo CSV
        df1 = pd.read_csv(csv_path)

        # Generar el informe Sweetviz
        sweet_report = sv.analyze(df1)

        # Guardar el informe como HTML en la carpeta de reportes
        report_path = os.path.join(reports_folder, f"report_{csv_file}.html")
        sweet_report.show_html(report_path)

In [ ]:
#comprimimos la carpeta de reportes
import shutil

shutil.make_archive("/content/reportes", 'zip', "/content/Reviews_estados")

'/content/reportes.zip'

In [ ]:
#comprimimos la carpeta de reportes
import shutil

shutil.make_archive("/content/reportes", 'zip', "/content/Reviews_estados")
#descargamos la carpeta comprimida
from google.colab import files

files.download("/content/reportes.zip")

'/content/reportes.zip'

In [ ]:
#descargamos la carpeta comprimida
from google.colab import files

files.download("/content/reportes.zip")

In [ ]:
#guardamos los nombres de las columnas a eliminar
b = ['description', 'price', 'MISC']

In [ ]:
#se clasifican los estados por punto cardinal
estados_norte = ['review-Maine','review-District_of_Columbia', 'review-New_Hampshire', 'review-Vermont', 'review-Massachusetts', 'review-Connecticut', 'review-Rhode_Island', 'review-New_York', 'review-Pennsylvania', 'review-New_Jersey']
estados_sur = ['review-Delaware', 'review-Maryland', 'review-Virginia', 'review-West_Virginia', 'review-North_Carolina', 'review-South_Carolina', 'review-Georgia', 'review-Florida', 'review-Alabama']
estados_sur1 =  ['review-Mississippi', 'review-Louisiana', 'review-Arkansas', 'review-Tennessee', 'review-Kentucky', 'review-Texas', 'review-Oklahoma']
estados_oeste = ['review-Hawaii', 'review-Alaska', 'review-Washington', 'review-Oregon', 'review-California', 'review-Nevada', 'review-Idaho', 'review-Utah', 'review-Arizona', 'review-Montana', 'review-Wyoming', 'review-Colorado', 'review-New_Mexico']
estados_este = ['review-Michigan', 'review-Ohio', 'review-Indiana', 'review-Illinois', 'review-Wisconsin', 'review-Minnesota', 'review-Iowa', 'review-Missouri', 'review-North_Dakota', 'review-South_Dakota', 'review-Nebraska', 'review-Kansas']

In [ ]:
#se guarda el directorio en una variable
directorio_drive = '/content/csvestados'

In [ ]:
# Creamos un DataFrame vacío para almacenar los datos de los estados del norte
df_norte = pd.DataFrame()

# Recorremos los archivos CSV en el directorio y cargamos los datos de los estados del norte
for estado in estados_norte:
    archivo_csv = os.path.join(directorio_drive, f'{estado}.csv')
    if os.path.exists(archivo_csv):
        df_estado = pd.read_csv(archivo_csv)
        df_norte = pd.concat([df_norte, df_estado], ignore_index=True)

# Guardamos el DataFrame resultante en un archivo 'usa-norte.csv'
df_norte.to_csv(os.path.join(directorio_drive, 'usa-norte.csv'), index=False)

print("Los estados del norte de USA se han agrupado en 'usa-norte.csv'.")

Los estados del norte de USA se han agrupado en 'usa-norte.csv'.


In [ ]:
# Creamos un DataFrame vacío para almacenar los datos de los estados del sur
df_sur = pd.DataFrame()

# Recorremos los archivos CSV en el directorio y cargamos los datos de los estados del sur
for estado in estados_sur:
    archivo_csv = os.path.join(directorio_drive, f'{estado}.csv')
    if os.path.exists(archivo_csv):
        df_estado = pd.read_csv(archivo_csv)
        df_sur = pd.concat([df_sur, df_estado], ignore_index=True)

# Guardamos el DataFrame resultante en un archivo 'usa-sur.csv'
df_sur.to_csv(os.path.join(directorio_drive, 'usa-sur.csv'), index=False)

print("Los estados del sur de USA se han agrupado en 'usa-sur.csv'.")

Los estados del sur de USA se han agrupado en 'usa-sur.csv'.


In [ ]:
# Creamos un DataFrame vacío para almacenar los datos de los estados del sur1
df_sur1 = pd.DataFrame()

# Recorremos los archivos CSV en el directorio y cargamos los datos de los estados del sur1
for estado in estados_sur1:
    archivo_csv = os.path.join(directorio_drive, f'{estado}.csv')
    if os.path.exists(archivo_csv):
        df_estado = pd.read_csv(archivo_csv)
        df_sur1 = pd.concat([df_sur1, df_estado], ignore_index=True)

# Guardamos el DataFrame resultante en un archivo 'usa-sur1.csv'
df_sur1.to_csv(os.path.join(directorio_drive, 'usa-sur1.csv'), index=False)

print("Los estados del sur1 de USA se han agrupado en 'usa-sur1.csv'.")

Los estados del sur1 de USA se han agrupado en 'usa-sur1.csv'.


In [ ]:
# Creamos un DataFrame vacío para almacenar los datos de los estados del este
df_este = pd.DataFrame()

# Recorremos los archivos CSV en el directorio y cargamos los datos de los estados del este
for estado in estados_este:
    archivo_csv = os.path.join(directorio_drive, f'{estado}.csv')
    if os.path.exists(archivo_csv):
        df_estado = pd.read_csv(archivo_csv)
        df_este = pd.concat([df_este, df_estado], ignore_index=True)

# Guardamos el DataFrame resultante en un archivo 'usa-este.csv'
df_este.to_csv(os.path.join(directorio_drive, 'usa-este.csv'), index=False)

print("Los estados del este de USA se han agrupado en 'usa-este.csv'.")

Los estados del este de USA se han agrupado en 'usa-este.csv'.


In [ ]:
# Creamos un DataFrame vacío para almacenar los datos de los estados del oeste
df_oeste = pd.DataFrame()

# Recorremos los archivos CSV en el directorio y cargamos los datos de los estados del oeste
for estado in estados_oeste:
    archivo_csv = os.path.join(directorio_drive, f'{estado}.csv')
    if os.path.exists(archivo_csv):
        df_estado = pd.read_csv(archivo_csv)
        df_oeste = pd.concat([df_oeste, df_estado], ignore_index=True)

# Guardamos el DataFrame resultante en un archivo 'usa-oeste.csv'
df_oeste.to_csv(os.path.join(directorio_drive, 'usa-oeste.csv'), index=False)

print("Los estados del oeste de USA se han agrupado en 'usa-oeste.csv'.")

Los estados del oeste de USA se han agrupado en 'usa-oeste.csv'.


In [ ]:
#despues de obtener los archivos de los puntos cardinales, eliminamos los csv de los estados individuales manualmente
#copiamos los archivos csv resultantes a google drive
import shutil

origen = '/content/csvestados'  # Ruta de la carpeta en Colab
destino = '/content/drive/My Drive/'  # Ruta en tu Google Drive

shutil.copytree(origen, destino + 'mi_carpeta')

'/content/drive/My Drive/mi_carpeta'

ARCHIVOS METADATA-SITIOS

In [ ]:
#en esta columna se presenta un error en el formato de todos los archivos json, se decide guardar en una variable para ser eliminada más adelante
a= ['url']

LECTURA DE ARCHIVOS Y UNION

In [ ]:
uno=pd.read_json('/content/drive/MyDrive/Google_Maps/metadata-sitios/1.json',lines=True)

In [ ]:
dos=pd.read_json('/content/drive/MyDrive/Google_Maps/metadata-sitios/2.json',lines=True)

In [ ]:
tres=pd.read_json('/content/drive/MyDrive/Google_Maps/metadata-sitios/3.json',lines=True)

In [ ]:
cuatro=pd.read_json('/content/drive/MyDrive/Google_Maps/metadata-sitios/4.json',lines=True)

In [ ]:
cinco=pd.read_json('/content/drive/MyDrive/Google_Maps/metadata-sitios/5.json',lines=True)

In [ ]:
seis=pd.read_json('/content/drive/MyDrive/Google_Maps/metadata-sitios/6.json',lines=True)

In [ ]:
siete=pd.read_json('/content/drive/MyDrive/Google_Maps/metadata-sitios/7.json',lines=True)

In [ ]:
ocho=pd.read_json('/content/drive/MyDrive/Google_Maps/metadata-sitios/8.json',lines=True)

In [ ]:
#se separan los archivos para mejor control de la ram de la maquina virtual colab
file_list = [uno,dos,tres,cuatro,cinco,seis,siete,ocho]

In [ ]:
# Inicializamos una lista para almacenar los DataFrames individuales
data_frames = []

In [ ]:
# Iteramos a través de los archivos JSON y cargamos cada uno en un DataFrame
for file in file_list:
    data = file
    df = pd.DataFrame(data)
    df.drop(a, axis=1)
    data_frames.append(df)

# Combinamos los DataFrames en uno solo utilizando pd.concat
combined_data = pd.concat(data_frames, ignore_index=True)

In [ ]:
#exportamos como archivo csv
combined_data.to_csv("1-8.csv", index=False)

In [ ]:
#leemos los archivos restantes guardandolos en una variable individual siguiendo el mismo proceso
nueve=pd.read_json('/content/drive/MyDrive/Google_Maps/metadata-sitios/9.json',lines=True)

In [ ]:
diez=pd.read_json('/content/drive/MyDrive/Google_Maps/metadata-sitios/10.json',lines=True)

In [ ]:
once=pd.read_json('/content/drive/MyDrive/Google_Maps/metadata-sitios/11.json',lines=True)

In [ ]:
file_list_2 = [nueve,diez,once]

In [ ]:
# Inicializamos una lista para almacenar los DataFrames individuales
data_frames = []

In [ ]:
# Iteramos a través de los archivos JSON y carga cada uno en un DataFrame
for file in file_list_2:
    data = file
    df = pd.DataFrame(data)
    df.drop(a, axis=1)
    data_frames.append(df)

# Combinamos los DataFrames en uno solo utilizando pd.concat
combined_data1 = pd.concat(data_frames, ignore_index=True)

In [ ]:
#exportamos como archivo csv
combined_data1.to_csv("91011.csv", index=False)

UNION DE ARCHIVOS CSV ANTERIORES Y LIMPIEZA

In [ ]:
#leemos el csv que contiene los archivos json del 9 al 11
met1 = pd.read_csv('/content/91011.csv')

In [ ]:
#columnas a eliminar
col=['description', 'price', 'MISC']

In [ ]:
#eliminamos columnas
met1 = met1.drop(col, axis= 1)

In [ ]:
#consultamos tipos de datos por columna
met1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825003 entries, 0 to 825002
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              824994 non-null  object 
 1   address           809470 non-null  object 
 2   gmap_id           825003 non-null  object 
 3   latitude          825003 non-null  float64
 4   longitude         825003 non-null  float64
 5   category          821563 non-null  object 
 6   avg_rating        825003 non-null  float64
 7   num_of_reviews    825003 non-null  int64  
 8   hours             642950 non-null  object 
 9   state             653815 non-null  object 
 10  relative_results  766544 non-null  object 
 11  url               825003 non-null  object 
dtypes: float64(3), int64(1), object(8)
memory usage: 75.5+ MB


In [ ]:
#revisamos nulos
met1.isnull().sum()

name                     9
address              15533
gmap_id                  0
latitude                 0
longitude                0
category              3440
avg_rating               0
num_of_reviews           0
hours               182053
state               171188
relative_results     58459
url                      0
dtype: int64

In [ ]:
#revisamos duplicados
met1.duplicated().sum()

0

In [ ]:
#eliminamos duplicados conservando el primer valor
met1 = met1.drop_duplicates(keep="first")

In [ ]:
#revisamos cambios
met1.head(3)

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,hours,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,['Pharmacy'],4.9,16,"[['Friday', '8AM–6PM'], ['Saturday', '8AM–12PM...",Open ⋅ Closes 6PM,"['0x88f16e41929435cf:0x5b2532a2885e9ef6', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
1,MemorialCare Breast Center - Irvine Sand Canyon,MemorialCare Breast Center - Irvine Sand Canyo...,0x80dcdd3b95c7284b:0x23d9b4440c982d37,33.668389,-117.765649,['Mammography service'],4.7,33,"[['Tuesday', '8:30AM–5PM'], ['Wednesday', '8:3...",Closed ⋅ Opens 8:30AM,"['0x80dcdc4aad8cb91f:0x62bdeb79a44d7633', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
2,U-Haul Neighborhood Dealer,"U-Haul Neighborhood Dealer, 1690 E 112th Ave, ...",0x876c769e600eda1d:0xdf1fffc2e00e1fe1,39.899189,-104.966765,"['Truck rental agency', 'Trailer rental servic...",3.9,18,NaN,NaN,"['0x876c75d05d0e1bf3:0x1d2d9897d0dec9cb', '0x8...",https://www.google.com/maps/place//data=!4m2!3...


In [ ]:
#leemos csv con los archivos json del 1 al 8
met = pd.read_csv('/content/1-8.csv')

In [ ]:
#eliminamos columnas anteriormente guardadas en una variable
met = met.drop(col, axis= 1)

In [ ]:
#consultamos nulos
met.isnull().sum()

name                    41
address              64978
gmap_id                  0
latitude                 0
longitude                0
category             13979
avg_rating               0
num_of_reviews           0
hours               605352
state               575267
relative_results    236599
url                      0
dtype: int64

In [ ]:
#consultamos duplicados
met.duplicated().sum()

26580

In [ ]:
#eliminamos duplicados
met = met.drop_duplicates(keep="first")

In [ ]:
#revisamos cambios
met.head(3)

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,hours,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.35710,['Pharmacy'],4.9,16,"[['Friday', '8AM–6PM'], ['Saturday', '8AM–12PM...",Open ⋅ Closes 6PM,"['0x88f16e41929435cf:0x5b2532a2885e9ef6', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.21529,['Textile exporter'],4.5,6,NaN,Open now,"['0x80c2c624136ea88b:0xb0315367ed448771', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.29213,['Korean restaurant'],4.4,18,"[['Thursday', '6:30AM–6PM'], ['Friday', '6:30A...",Open ⋅ Closes 6PM,"['0x80c2c78249aba68f:0x35bf16ce61be751d', '0x8...",https://www.google.com/maps/place//data=!4m2!3...


In [ ]:
# Concatenamos los DataFrames
concatenated_df = pd.concat([met, met1], ignore_index=True)  # ignore_index=True reinicia los índices


In [ ]:
#consultamos cantidad de registros
concatenated_df

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,hours,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,['Pharmacy'],4.9,16,"[['Friday', '8AM–6PM'], ['Saturday', '8AM–12PM...",Open ⋅ Closes 6PM,"['0x88f16e41929435cf:0x5b2532a2885e9ef6', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.215290,['Textile exporter'],4.5,6,NaN,Open now,"['0x80c2c624136ea88b:0xb0315367ed448771', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,['Korean restaurant'],4.4,18,"[['Thursday', '6:30AM–6PM'], ['Friday', '6:30A...",Open ⋅ Closes 6PM,"['0x80c2c78249aba68f:0x35bf16ce61be751d', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,34.023669,-118.232930,['Fabric store'],3.3,6,"[['Thursday', '9AM–5PM'], ['Friday', '9AM–5PM'...",Open ⋅ Closes 5PM,"['0x80c2c8811477253f:0x23a8a492df1918f7', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,34.036694,-118.249421,['Fabric store'],4.3,7,"[['Thursday', '9AM–5PM'], ['Friday', '9AM–5PM'...",Open ⋅ Closes 5PM,"['0x80c2c62c496083d1:0xdefa11317fe870a1', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...
2998424,Steak 48,"Steak 48, 260 S Broad St, Philadelphia, PA 19102",0x89c6c74f43a49b55:0x6be6995921c58b12,39.947254,-75.164953,"['Steak house', 'Fine dining restaurant', 'Sea...",4.6,308,"[['Monday', '4–10PM'], ['Tuesday', '4–10PM'], ...",Closed ⋅ Opens 4PM,"['0x89c6c7af1a55f4f9:0x589bb93addf7fd95', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
2998425,"Jack Mcnerney Chevrolet, Inc.","Jack Mcnerney Chevrolet, Inc., 363 NY-281, Tul...",0x89da1787d3fdc4a5:0xce92ea6e3cd4d1cc,42.788636,-76.122120,['Chevrolet dealer'],4.4,143,"[['Monday', '8:30AM–7PM'], ['Tuesday', '8:30AM...",Open ⋅ Closes 7PM,"['0x89d98d228469e435:0x3fdc4fa72594151', '0x89...",https://www.google.com/maps/place//data=!4m2!3...
2998426,Central Ny Spay Neuter Assista,"Central Ny Spay Neuter Assista, 17 Salisbury S...",0x89da6b9ce832726f:0xbd0d3630f1e3b42e,42.603831,-76.165286,['Veterinarian'],4.3,88,NaN,Open now,"['0x89da6c67512700cb:0x69892b3a9a648efe', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
2998427,Ok Feed Store,"Ok Feed Store, 16300 SW 296th St, Homestead, F...",0x88d9e872a9e39f3d:0x5de29f50507e5d73,25.491641,-80.456076,['Animal feed store'],4.9,13,"[['Monday', '9AM–6PM'], ['Tuesday', '9AM–6PM']...",Open ⋅ Closes 6PM,"['0x88d9e86cb955da7b:0xeda838b28926c524', '0x8...",https://www.google.com/maps/place//data=!4m2!3...


In [ ]:
#verificamos de nuevo la eliminacion de duplicados
concatenated_df = concatenated_df.drop_duplicates(keep="first")

In [ ]:
# Exportamos el DataFrame concatenado a un archivo CSV
concatenated_df.to_csv('metadt.csv', index=False)

In [ ]:
source_path = '/content/metadt.csv'  # Ruta del archivo en Colab
destination_path = '/content/drive/MyDrive'  # Ruta de destino en Google Drive
#guardamos el archivo en google drive
shutil.copy(source_path, destination_path)

'/content/drive/MyDrive/metadt.csv'

In [ ]:
#leemos el archivo concatenado
metdt = pd.read_csv('/content/drive/MyDrive/metadt.csv')

In [ ]:
#revisamos
metdt

In [ ]:
#cambiamos la columna category a tipo de dato str
metdt['category'] = metdt['category'].astype(str)


In [ ]:
#creamos funcion para sacar el primer valor de las listas que hay en la columna category
def desanidar_columna(columna):
  """
  Desanida una columna de texto.

  Args:
    columna: La columna de texto a desanidar.

  Returns:
    La columna desanidada.
  """

  # Obtiene la primera parte de cada texto.
  primera_parte = columna.str.split(",").str[0]

  # Devuelve la columna desanidada.
  return primera_parte

In [ ]:
#limpiamos los restos como '[]' o ',' para obtener solo el texto
def limpiar_columna(columna):
  """
  Limpia una columna de texto.

  Args:
    columna: La columna de texto a limpiar.

  Returns:
    La columna limpia.
  """

  # Elimina los caracteres especiales.
  limpia = columna.str.replace("[", "").str.replace("]", "").str.replace("'", "")

  # Devuelve la columna limpia.
  return limpia

In [ ]:
# Desanida la columna.
metdt["category"] = desanidar_columna(metdt["category"])


In [ ]:
#limpiamos de caracteres especiales
metdt["category"] = limpiar_columna(metdt["category"])

<ipython-input-16-977794adcbd3>:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  limpia = columna.str.replace("[", "").str.replace("]", "").str.replace("'", "")


In [ ]:
#revisamos cambios
metdt